# Provider Fraud Detection - Data Wrangling

In [1]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySparkTest") \
    .master("local[*]") \
    .getOrCreate()

# Create a simple DataFrame
data = [("Alice", 34), ("Bob", 45), ("Cathy", 29)]
columns = ["Name", "Age"]
df = spark.createDataFrame(data, columns)

# Show the DataFrame
df.show()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/25 18:52:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/25 18:52:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+---+
| Name|Age|
+-----+---+
|Alice| 34|
|  Bob| 45|
|Cathy| 29|
+-----+---+



In [2]:
print(spark.sparkContext.version)


4.0.0


In [ ]:
# Load a large dataset from a subfolder and show a few rows
df_large = spark.read.csv("CMS_Physician_Supplier_Procedure_Summary/Physician_Supplier_Procedure_Summary.csv", header=True, inferSchema=True)
df_large.show(5)  # Show first 5 rows

+--------+-------------------------+----------------+-----------+-------------------+------------------+-------------------+------------------------+--------------------------+-------------------------+-----------------------+------------------------+----------------------+--------------------------+--------------------+---------------------+-----------------+--------------+
|HCPCS_CD|HCPCS_INITIAL_MODIFIER_CD|PROVIDER_SPEC_CD|CARRIER_NUM|PRICING_LOCALITY_CD|TYPE_OF_SERVICE_CD|PLACE_OF_SERVICE_CD|HCPCS_SECOND_MODIFIER_CD|PSPS_SUBMITTED_SERVICE_CNT|PSPS_SUBMITTED_CHARGE_AMT|PSPS_ALLOWED_CHARGE_AMT|PSPS_DENIED_SERVICES_CNT|PSPS_DENIED_CHARGE_AMT|PSPS_ASSIGNED_SERVICES_CNT|PSPS_NCH_PAYMENT_AMT|PSPS_HCPCS_ASC_IND_CD|PSPS_ERROR_IND_CD|HCPCS_BETOS_CD|
+--------+-------------------------+----------------+-----------+-------------------+------------------+-------------------+------------------------+--------------------------+-------------------------+-----------------------+------------------